In [ ]:
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (6, 6)

import scipy.ndimage as ndimage

import math
import numpy as np
np.set_printoptions(threshold=np.nan)
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm

from astropy.io import fits
import os.path

import ipywidgets
from ipywidgets import interact

import sys
sys.path.append("../")
sys.path.append("../utils")

import datapipe
import datapipe.image.kill_isolated_pixels as kill_islands
import common_functions as common

In [ ]:
file_path_list = [
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV155601.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV193406.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV347307.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV427404.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV450207.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV454807.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV48104.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV58000.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV126402.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV131305.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV182506.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV225002.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV26603.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV458108.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV483109.fits",
    "/Users/jdecock/data/astri_mini_array/fits/gamma/run1001.simtel.gz_TEL001_EV62605.fits"
    ]

## Datapipe's implementation

In [ ]:
@interact(file_path=file_path_list, threshold=0.2)
def compute_hillas_and_display(file_path, threshold):
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(16, 8))

    images_dict, metadata_dict = datapipe.io.images.load_benchmark_images(file_path)
    
    img = images_dict["reference_image"]
    pixels_positions = images_dict["pixels_position"]

    # GET ISLANDS ##############################################
    
    filtered_img, islands, num_labels = datapipe.image.kill_isolated_pixels.get_islands(img, threshold=0.2)
    common.plot_image_meter(ax1, islands, pixels_positions, title="get_islands()", plot_log_scale=False)
    common.plot_image_meter(ax2, filtered_img, pixels_positions, title="get_islands()", plot_log_scale=True)
    #print("filtered_img:", filtered_img)       # may contain NaN
    #print("islands:", islands)                 # doesn't contain NaN
    print("num_labels:", num_labels)
    
    # KILL ISOLATED PIXELS #####################################
    
    cleaned_img = datapipe.image.kill_isolated_pixels.kill_isolated_pixels(img, threshold=0.2)
    common.plot_image_meter(ax3, cleaned_img, pixels_positions, title="kill_isolated_pixels()", plot_log_scale=True)
    #print("cleaned_img:", cleaned_img)         # may contain NaN

    # KILL ISOLATED PIXELS STATS ###############################
    
    stats = datapipe.image.kill_isolated_pixels.kill_isolated_pixels_stats(img, threshold=0.2)
    print("kill_isolated_pixels_stats():", stats)
    
    # NUMBER OF ISLANDS ########################################
    
    num_islands = datapipe.image.kill_isolated_pixels.number_of_islands(img, threshold=0.2)
    print("number_of_islands():", num_islands)

## Basic implementation (doesn't handle NaN)

In [ ]:
@interact(file_path=file_path_list, threshold=0.2)
def compute_hillas_and_display(file_path, threshold):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 16))

    images_dict, metadata_dict = datapipe.io.images.load_benchmark_images(file_path)
    
    array = images_dict["reference_image"]

    # KILL ISLANDS #############################################
    
    filtered_array = np.copy(array)

    # Put to 0 pixels that are below 'threshold'
    if threshold is not None:
        filtered_array[filtered_array < threshold] = 0
    mask = filtered_array > 0

    # Detect islands ("label")
    label_array, num_labels = ndimage.label(mask)#, structure=np.ones((5, 5)))

    # Count the number of pixels for each island
    num_pixels_per_island = ndimage.sum(filtered_array, label_array, range(num_labels + 1))

    # Only keep the biggest island
    mask_biggest_island = num_pixels_per_island < np.max(num_pixels_per_island)
    remove_pixel = mask_biggest_island[label_array]

    filtered_array[remove_pixel] = 0
    
    # PLOT #####################################################
    
    pixels_positions = images_dict["pixels_position"]
    common.plot_image_meter(ax1, array, pixels_positions, title="Not killed", plot_log_scale=True)
    common.plot_image_meter(ax2, label_array, pixels_positions, title="Labels", plot_log_scale=False)
    common.plot_image_meter(ax3, filtered_array, pixels_positions, title="Killed", plot_log_scale=True)
    
    print("Num labels:", num_labels)
    print("Num pixels per island:", np.sort(num_pixels_per_island))
    
    print("NPE of the filtered image (i.e. of the biggest island):", np.sum(filtered_array))

## NaN compliant implementation

In [ ]:
@interact(file_path=file_path_list, threshold=0.2)
def compute_hillas_and_display(file_path, threshold):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 16))

    images_dict, metadata_dict = datapipe.io.images.load_benchmark_images(file_path)
    
    array = images_dict["reference_image"]

    assert threshold >= 0
    
    # KILL ISLANDS #############################################
    
    filtered_array = np.copy(array)
    
    # Put NaN pixels to 0
    filtered_array[np.isnan(filtered_array)] = 0

    # Put to 0 pixels that are below 'threshold'
    if threshold is not None:
        filtered_array[filtered_array < threshold] = 0
    mask = filtered_array > 0
    
    # mask is a boolean 2d array
    #print("mask:", mask)

    # Detect islands ("label")
    label_array, num_labels = ndimage.label(mask)#, structure=np.ones((5, 5)))
    
    #print("label_array:", label_array)
    print("num_labels:", num_labels)

    # Count the number of pixels for each island
    num_pixels_per_island = ndimage.sum(filtered_array, label_array, range(num_labels + 1))

    # Only keep the biggest island
    mask_biggest_island = num_pixels_per_island < np.max(num_pixels_per_island)
    remove_pixel = mask_biggest_island[label_array]

    filtered_array[remove_pixel] = 0
    #print(filtered_array)
    
    filtered_array[np.isnan(array)] = np.nan
    #print(filtered_array)
    
    # PLOT #####################################################
    
    pixels_positions = images_dict["pixels_position"]
    common.plot_image_meter(ax1, array, pixels_positions, title="Not killed", plot_log_scale=True)
    common.plot_image_meter(ax2, label_array, pixels_positions, title="Labels", plot_log_scale=False)
    common.plot_image_meter(ax3, filtered_array, pixels_positions, title="Killed", plot_log_scale=True)
    
    print("Num pixels per island:", np.sort(num_pixels_per_island))
    
    print("NPE of the filtered image (i.e. of the biggest island):", np.sum(filtered_array))